In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment

C:\Users\deepa\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
def eucl_dist(vec1, vec2):
    minn_length = min(vec1.shape[1], vec2.shape[1])
    if vec1.shape[1] < vec2.shape[1]: 
        vec2 = vec2[:,:minn_length]
    else: 
        vec1 = vec1[:,:minn_length]
    return np.linalg.norm(vec1-vec2)

In [13]:
# PROPERTY OF DEEPMIND1234
def extract_features(file_path):
    # Load the audio file
    y, sr = librosa.load(file_path, sr=22050)

    # Extract the MFCC features
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    # Extract the chroma features
    chroma = librosa.feature.chroma_cqt(y=y, sr=sr)

    # Extract the spectral contrast features
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)

    # Concatenate the features into a single array
    features = np.concatenate([mfcc, chroma, contrast, spectral_centroid, zero_crossing_rate], axis=0)

    return features

In [14]:
features_d = extract_features('media/deepak_test_1.wav')
features_a = extract_features('media/aadarsh_test_1.wav')
features_s = extract_features('media/sean_test_1.wav')
features_ta = extract_features('media/test_vector.wav')

In [15]:
print(eucl_dist(features_d, features_a))
print(eucl_dist(features_d, features_s))
print(eucl_dist(features_s, features_a))

10486.538379977239
12928.7697877973
14892.044959115241


In [16]:
print(eucl_dist(features_d, features_ta))
print(eucl_dist(features_a, features_ta))
print(eucl_dist(features_s, features_ta))

16792.012428447484
17102.959813213223
21862.665378129223


In [17]:
feature_a_new = extract_features('media/aadarsh_inference.wav')
feature_dict = {"deepak":features_d,"aadarsh":features_a,"sean":features_s,"TA":features_ta}

def NearestNeighbour(test_features,voice_set):
    f_min = 10e20
    similar = None
    for key,value in voice_set.items():
        cur_min = eucl_dist(test_features,value)
        print(str(key) + ":"+ str(cur_min))
        if cur_min < f_min:
            f_min = cur_min
            similar = key
    return similar

x = NearestNeighbour(feature_a_new,feature_dict)
print(x)        
    

deepak:21169.52388607438
aadarsh:19125.634532576056
sean:25665.735956872362
TA:22200.584062091377
aadarsh
